In [ ]:
import pandas as pd
import numpy as np
import pickle

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from nltk.corpus import stopwords
from textblob import TextBlob
import spacy
from collections import Counter
import re
import time
import requests
from bs4 import BeautifulSoup
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


In [ ]:
# load data

# train = pd.read_csv('../data/train2.tsv', sep='\t', header=None)
# test = pd.read_csv('../data/test2.tsv', sep='\t', header=None)
data = pd.read_csv('../data/raw/train2.tsv', sep='\t', header=None)
column_names = [
    "Index",
    "ID",
    "Label",
    "Statement",
    "Subject",
    "Speaker",
    "Speaker_Job_Title",
    "State_Info",
    "Party_Affiliation",
    "Barely_True_Counts",
    "False_Counts",
    "Half_True_Counts",
    "Mostly_True_Counts",
    "Pants_On_Fire_Counts",
    "Context",
    "Extracted_Justification"
]

# train.columns = column_names
# test.columns = column_names
data.columns = column_names

In [ ]:
def data_clean(df):

    # Fill missing values in text-based columns
    df.replace({'Statement':''}, np.nan, inplace=True)
    df.replace({'Extracted_Justification':''}, np.nan, inplace=True)

    # Drop rows with missing 'Statement' or 'Extracted_Justification' (since these are critical)
    df.dropna(subset=['Statement', 'Extracted_Justification'], inplace=True)

    # Drop rows with missing labels
    df = df.dropna(subset=['Label'])

    # Impute missing values in categorical columns with 'Unknown'
    categorical_columns = ['Speaker', 'Speaker_Job_Title', 'State_Info', 'Party_Affiliation', 'Context']
    df[categorical_columns] = df[categorical_columns].fillna('Unknown')

    # Impute numerical columns (truth counts) with median values
    numeric_columns = ["Barely_True_Counts", "False_Counts", "Half_True_Counts", "Mostly_True_Counts", "Pants_On_Fire_Counts"]
    df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')
    df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].median())
    return df

# Check for missing values
print(data.isnull().sum())

data = data_clean(data)

print(data.isnull().sum())

In [ ]:
data.head()

In [ ]:
data['Label']

In [ ]:
# Label Encoding for categorical variables
# {barely-true: 0, false:1, half-true:2, mostly-true:3, pants-fire:4, true:5 }

label_encoder = LabelEncoder()

label_encoder.fit(data['Label'])
le_name_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
data['Label'] = label_encoder.fit_transform(data['Label']) 

X = data.drop(['Label'], axis=1)
y = data['Label']

In [ ]:
# Load a pre-trained NLP model
nltk.download('punkt', quiet=True)  # Tokenizer
nltk.download('stopwords', quiet=True)
stop_words = set(stopwords.words('english'))
nltk.download('averaged_perceptron_tagger')  # POS tagger
nltk.download('wordnet')  # WordNet for lemmatization
nlp = spacy.load("en_core_web_sm")

## Content Statistic

In [ ]:
# FF: content statistic

def structural_analysis(statement):
    doc = nlp(statement)
    num_sentences = len(list(doc.sents))
    total_tokens = len([token.text for token in doc])
    
    # Syntactic complexity
    avg_sentence_length = total_tokens / num_sentences if num_sentences > 0 else 0
    tree_depth = max([token.head.i - token.i for token in doc]) if len(doc) > 0 else 0
    
    # Sentiment Analysis
    sentiment = TextBlob(statement).sentiment
    polarity = sentiment.polarity
    subjectivity = sentiment.subjectivity

    return [avg_sentence_length, tree_depth, polarity, subjectivity]

def extract_graph_features(statement):
    doc = nlp(statement)
    pos_counts = Counter([token.pos_ for token in doc])
    entities = Counter([ent.label_ for ent in doc.ents])
    
    # part of speech tagging
    pos_noun = pos_counts.get("NOUN", 0)
    pos_verb = pos_counts.get("VERB", 0)
    pos_adjective = pos_counts.get("ADJ", 0)
    
    # named entity recognition
    num_persons = entities.get("PERSON", 0)
    num_orgs = entities.get("ORG", 0)
    num_gpes = entities.get("GPE", 0)
    
    return [pos_noun, pos_verb, pos_adjective, num_persons, num_orgs, num_gpes]

def extract_comparison_features(statement):
    # Keywords for different LIWC-like categories (simplified)
    cognitive_words = ["think", "know", "understand", "believe"]
    emotional_words = ["happy", "sad", "angry", "fear"]
    social_words = ["friend", "family", "society"]

    # Tokenize statement and count keywords
    vectorizer = CountVectorizer(vocabulary=cognitive_words + emotional_words + social_words)
    word_counts = vectorizer.fit_transform([statement]).toarray().flatten()

    # Divide word counts into different categories
    num_cognitive = sum(word_counts[:len(cognitive_words)])
    num_emotional = sum(word_counts[len(cognitive_words):len(cognitive_words) + len(emotional_words)])
    num_social = sum(word_counts[-len(social_words):])

    return [num_cognitive, num_emotional, num_social]

    

In [ ]:
def extract_feature(statement):
    return np.array(structural_analysis(statement) + extract_graph_features(statement) + extract_comparison_features(statement))

## Authenticity

In [ ]:
# Function to scrape one page of fact-checks
def scrape_fact_checks(page_number):
    url = f"https://www.politifact.com/factchecks/?page={page_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # List to store the fact-check data
    data = []

    # Find all fact-check articles on the page
    fact_checks = soup.find_all('article', class_='m-statement')

    for fact in fact_checks:
        # Extract Author/Speaker
        author = fact.find('a', class_='m-statement__name').text.strip()

        # Extract the Date of the statement
        date_string = fact.find('div', class_='m-statement__desc').text.strip()

        # Use a regular expression to extract only the date portion (e.g., October 8, 2024)
        date_match = re.search(r'([A-Za-z]+ \d{1,2}, \d{4})', date_string)
        date = date_match.group(0) if date_match else "No date found"

        # Extract the Claim (statement being fact-checked)
        claim = fact.find('div', class_='m-statement__quote').find('a').text.strip()

        # Extract the URL to the full fact-check article
        link = "https://www.politifact.com" + fact.find('div', class_='m-statement__quote').find('a')['href']

        # Extract the Rating (e.g., False, Pants on Fire)
        rating = fact.find('div', class_='m-statement__meter').find('img')['alt'].strip()

        # Append the extracted information to the list
        data.append({
            'Author/Speaker': author,
            'Date': date,
            'Claim': claim,
            'Rating': rating,
            'Link to Full Article': link
        })

    return data

# Loop through multiple pages and collect data
def scrape_multiple_pages(start_page, end_page):
    all_data = []
    for page_number in range(start_page, end_page + 1):
        print(f"Scraping page {page_number}...")
        page_data = scrape_fact_checks(page_number)
        all_data.extend(page_data)
        time.sleep(2)  # Sleep for 2 seconds between each page request

    return all_data

# Scrape data from page 1 to 2
data = scrape_multiple_pages(1, 2)
politifact_data = pd.DataFrame(data)
test_link = politifact_data['Link to Full Article'].iloc[0]

test_response = requests.get(test_link)
soup = BeautifulSoup(test_response.text, 'html.parser')

In [ ]:
# FF: authenticity

def cross_referenced(x, politifact_data):
    # Get the 'statement' column from the Liar Plus dataset and 'Claim' from PolitiFact
    liar_statements = x['Statement']
    politifact_claims = politifact_data['Claim']

    # Use TF-IDF Vectorizer to convert text to numerical features
    vectorizer = TfidfVectorizer(stop_words='english')

    # Combine both statements and claims for vectorization
    combined_text = pd.concat([liar_statements, politifact_claims], axis=0)

    # Fit and transform the combined text using TF-IDF
    tfidf_matrix = vectorizer.fit_transform(combined_text)

    # Split the transformed matrix into two parts: one for Liar Plus, one for PolitiFact
    liar_tfidf = tfidf_matrix[:len(liar_statements)]
    politifact_tfidf = tfidf_matrix[len(liar_statements):]

    # Compute cosine similarity between every statement in Liar Plus and every claim in PolitiFact
    similarity_matrix = cosine_similarity(liar_tfidf, politifact_tfidf)

    # Find the highest similarity score for each Liar Plus statement
    max_similarity = similarity_matrix.max(axis=1)

    # Set a threshold for similarity (e.g., 0.8) to define a "cross-referenced" statement
    threshold = 0.8
    return (max_similarity >= threshold).astype(int)


In [ ]:
# Define a credibility score based on job title (you can customize this based on your data)
def assign_credibility_score(job_title):
    if "scientist" in job_title.lower() or "doctor" in job_title.lower():
        return 3  # High credibility
    elif "senator" in job_title.lower() or "president" in job_title.lower():
        return 2  # Medium credibility
    else:
        return 1  # Low credibility


In [ ]:
# Function to detect if justification contains references to studies or data
def contains_cited_data(justification):
    keywords = ['according to', 'research', 'study', 'data', 'shown by', 'reported']
    for keyword in keywords:
        if keyword in justification.lower():
            return 1  # Cited data present
    return 0  # No cited data

## Linguistic based, Toxicity

In [ ]:
sid = SentimentIntensityAnalyzer()

# Vectorizer
vectorizer = CountVectorizer(stop_words='english', max_features=50)

def fit_vectorizer(X):
    """Fit the vectorizer on the 'Statement' column of X."""
    vectorizer.fit(X['Statement'])

def transform_data(X):
    """Transform the data and generate the feature set."""
    features = []
    
    # Token/word count for normalization
    X['chunk_length'] = X['Statement'].apply(lambda x: len(x.split()))

    # Writing style and linguistic features
    X['exclamation_count'] = X['Statement'].apply(lambda x: x.count('!')) / X['chunk_length']
    X['question_mark_count'] = X['Statement'].apply(lambda x: x.count('?')) / X['chunk_length']
    X['all_caps_count'] = X['Statement'].apply(lambda x: len(re.findall(r'\b[A-Z]{2,}\b', x))) / X['chunk_length']
    
    # Named Entity Count: Counts the number of named entities in the statement using spaCy
    X['entity_count'] = X['Statement'].apply(lambda x: len(nlp(x).ents)) / X['chunk_length']
    
    # Superlative and adjective count using spaCy
    X['superlative_count'] = X['Statement'].apply(lambda x: len(re.findall(r'\b(best|worst|most|least)\b', x.lower()))) / X['chunk_length']
    X['adjective_count'] = X['Statement'].apply(lambda x: len([token for token in nlp(x) if token.pos_ == 'ADJ'])) / X['chunk_length']

    # Emotion-based word count
    emotion_words = set(['disaster', 'amazing', 'horrible', 'incredible', 'shocking', 'unbelievable'])
    X['emotion_word_count'] = X['Statement'].apply(lambda x: len([word for word in x.lower().split() if word in emotion_words])) / X['chunk_length']

    # Modal verb count
    modal_verbs = set(['might', 'could', 'must', 'should', 'would', 'may'])
    X['modal_verb_count'] = X['Statement'].apply(lambda x: len([word for word in x.lower().split() if word in modal_verbs])) / X['chunk_length']

    # Complex word ratio
    X['complex_word_ratio'] = X['Statement'].apply(lambda x: len([word for word in x.split() if len(re.findall(r'[aeiouy]+', word)) > 2]) / (len(x.split()) + 1))

    # Sentiment analysis using VADER
    X['sentiment_polarity'] = X['Statement'].apply(lambda x: sid.polarity_scores(x)['compound'])
    X['sentiment_subjectivity'] = X['Statement'].apply(lambda x: sid.polarity_scores(x)['neu'])  # Using VADER's neutrality score

    # Flesch Reading Ease
    X['flesch_reading_ease'] = X['Statement'].apply(flesch_reading_ease)

    # Combine all features into a dataframe
    numerical_features = X[['exclamation_count', 'question_mark_count', 'all_caps_count',
                            'entity_count', 'superlative_count', 'adjective_count',
                            'emotion_word_count', 'modal_verb_count', 
                            'complex_word_ratio', 
                            'sentiment_polarity', 'sentiment_subjectivity', 'flesch_reading_ease']]
    
    features.append(numerical_features)
    return pd.concat(features, axis=1)

def flesch_reading_ease(text):
    """Compute Flesch Reading Ease score for readability analysis."""
    sentence_count = max(len(re.split(r'[.!?]+', text)), 1)
    word_count = len(text.split())
    syllable_count = sum([syllable_count_func(word) for word in text.split()])
    if word_count > 0:
        return (206.835 - 1.015 * (word_count / sentence_count) - 84.6 * (syllable_count / word_count)) / 100
    else:
        return 0

def syllable_count_func(word):
    """Count syllables in a word using regular expressions."""
    word = word.lower()
    syllables = re.findall(r'[aeiouy]+', word)
    return max(len(syllables), 1)


In [ ]:
# Combine all the engineered features
def combine_feat(X):
    X['cross_referenced'] = cross_referenced(X, politifact_data)  # Cross-referencing feature
    X['credibility_score'] = X['Speaker_Job_Title'].apply(assign_credibility_score)  # Author credentials feature
    X['cited_data'] = X['Extracted_Justification'].apply(contains_cited_data)  # Cited data verification feature
    X_auth = X[['cross_referenced', 'credibility_score', 'cited_data']].to_numpy()
    X_content = np.vstack(X['Statement'].apply(lambda x: extract_feature(x)).to_numpy())
    fit_vectorizer(X)
    X_ling_tox = transform_data(X).to_numpy()
    X = np.hstack((X_content, X_auth, X_ling_tox))
    return X
X = combine_feat(X)

In [ ]:
X

In [ ]:
# Initialize and train a Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X, y)

In [ ]:
# save trained model
filename = './data/processed/pred_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
# {barely-true: 0, false:1, half-true:2, mostly-true:3, pants-fire:4, true:5 }
# Predictions
y_pred = model.predict_proba(X)
weights = [0.4, 0.2, 0.6, 0.8, 0, 1]
weighted_data = y_pred*weights
scores = np.sum(weighted_data, axis=1)
len(scores)

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.predict_proba(X)

In [ ]:
np.sum(result*weights, axis=1)